In [1]:
import sys
import json
import base64
import serial
import serial.tools.list_ports
import threading
import time
import re
import io
import os
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QWidget, QPushButton, QLabel, QTextEdit,
    QVBoxLayout, QHBoxLayout, QFileDialog, QComboBox, QLineEdit, QMessageBox,
    QTabWidget, QGroupBox, QSpinBox, QProgressBar
)
from PyQt5.QtGui import QTextCursor, QColor, QTextCharFormat, QMovie
from PyQt5.QtCore import pyqtSignal, QObject, QTimer, Qt, QUrl
from PyQt5.QtWebEngineWidgets import QWebEngineView
import pyqtgraph as pg
import folium

class SerialReader(QObject):
    """
    Handles serial port communication, including reading data, sending commands,
    and managing OTA firmware updates.
    """
    data_received = pyqtSignal(str)
    command_sent = pyqtSignal()
    ota_progress = pyqtSignal(int) # Signal for OTA progress
    connection_status_changed = pyqtSignal(str) # Signal for connection status

    def __init__(self):
        super().__init__()
        self.ser = None
        self._running = False
        self.ota_in_progress = False # Flag to manage OTA state

    def connect(self, port, baudrate=115200):
        """
        Establishes a serial connection to the specified port.
        """
        try:
            self.ser = serial.Serial(port, baudrate, timeout=1)
            self._running = True
            # Start read loop in a separate daemon thread
            threading.Thread(target=self.read_loop, daemon=True).start()
            self.connection_status_changed.emit("connected") # Emit connected status
            return True
        except Exception as e:
            print(f"Error connecting: {e}")
            self.connection_status_changed.emit("disconnected") # Emit disconnected status
            return False

    def read_loop(self):
        """
        Continuously reads data from the serial port.
        """
        while self._running:
            if self.ser and self.ser.is_open and self.ser.in_waiting:
                try:
                    line = self.ser.readline().decode('utf-8', errors='ignore').strip()
                    if line:
                        self.data_received.emit(line)
                except serial.SerialException as e:
                    print(f"Serial read error: {e}")
                    self._running = False # Stop reading on error
                    self.connection_status_changed.emit("disconnected") # Emit disconnected status
                    break
            time.sleep(0.01) # Small delay to prevent busy-waiting

    def send(self, command):
        """
        Sends a command string to the serial port.
        """
        if self.ser and self.ser.is_open:
            try:
                self.ser.write((command + "\n").encode('utf-8'))
                self.command_sent.emit()
                return True
            except serial.SerialException as e:
                print(f"Serial write error: {e}")
                return False
        return False

    def send_file(self, filename):
        """
        Sends a file over serial, encoded in base64 and chunked for OTA updates.
        """
        if not (self.ser and self.ser.is_open):
            print("Not connected to serial port. Cannot send file.")
            self.ota_progress.emit(-1) # Indicate error
            return

        try:
            with open(filename, 'rb') as f:
                content = base64.b64encode(f.read()).decode('utf-8')
                
                # Send the initial OTA command with filename
                # Example: "OTA_START:firmware.bin"
                self.send(f"OTA_START:{os.path.basename(filename)}")
                time.sleep(0.5) # Give ESP32 time to prepare

                chunks = [content[i:i+256] for i in range(0, len(content), 256)] # Increased chunk size for efficiency
                total_chunks = len(chunks)
                
                for idx, chunk in enumerate(chunks):
                    label = f"OTA_CHUNK:{idx:04d}:" # Use 4 digits for chunk index
                    if not self.send(label + chunk):
                        print("Failed to send chunk, aborting file transfer.")
                        self.ota_progress.emit(-1)
                        break
                    progress = int((idx + 1) / total_chunks * 100)
                    self.ota_progress.emit(progress)
                    time.sleep(0.01) # Small delay between chunks, can be adjusted

                # Signal end of file transfer
                self.send("OTA_END") # Simplified end command
                self.command_sent.emit()
                self.ota_progress.emit(100) # Ensure 100% is emitted
                print("File transfer complete.")

        except Exception as e:
            print(f"Error sending file: {e}")
            self.ota_progress.emit(-1) # Indicate error

    def disconnect(self):
        """
        Closes the serial connection and stops the read loop.
        """
        self._running = False
        if self.ser and self.ser.is_open:
            self.ser.close()
        self.ota_in_progress = False
        self.connection_status_changed.emit("disconnected") # Emit disconnected status


class GCSWindow(QMainWindow):
    """
    Main application window for the Ground Control Station.
    Manages UI, serial communication, data plotting, and map display.
    """
    def __init__(self):
        super().__init__()
        self.setWindowTitle("🛰️ GCS ANTASENA - Full Features + SD Log + IMU + OTA (Animasi)")
        self.setGeometry(100, 100, 1300, 800)

        self.serial_reader = SerialReader()
        self.serial_reader.data_received.connect(self.handle_data)
        self.serial_reader.command_sent.connect(self.update_last_activity)
        self.serial_reader.ota_progress.connect(self.update_ota_progress)
        self.serial_reader.connection_status_changed.connect(self.update_animation_connection_status) # Connect new signal

        self.log = []
        self.auto_timer = QTimer()
        self.auto_timer.timeout.connect(lambda: self.serial_reader.send("1"))

        self.graphs = {}
        self.theme_dark = False

        self.map_lat = -6.9732
        self.map_lon = 107.6306

        # --- FIX: Initialize a global start time for all plots ---
        self.start_time = time.time() 

        # Deep Sleep Feature Variables
        self.deep_sleep_timer = QTimer()
        self.deep_sleep_timer.timeout.connect(self.check_deep_sleep_condition)
        self.last_activity_time = time.time()
        self.deep_sleep_timeout_seconds = 300
        self.deep_sleep_enabled = False # Track deep sleep monitoring state

        # LoRa Profiles
        self.lora_profiles = {
            "Default": {"sf": 12, "bw": 250000, "freq": 923000000, "tx": 16, "cr": 6, "preamble": 8},
            "Long Range (SF12)": {"sf": 12, "bw": 125000, "freq": 915000000, "tx": 20, "cr": 5, "preamble": 8},
            "High Bandwidth (BW500)": {"sf": 7, "bw": 500000, "freq": 915000000, "tx": 20, "cr": 5, "preamble": 8},
            "Local Test (SF7, Low Power)": {"sf": 7, "bw": 125000, "freq": 915000000, "tx": 10, "cr": 5, "preamble": 8},
            "Custom Profile A": {"sf": 10, "bw": 250000, "freq": 868000000, "tx": 18, "cr": 6, "preamble": 10},
            "Ultra Low Power (Long Preamble)": {"sf": 11, "bw": 62500, "freq": 433000000, "tx": 15, "cr": 8, "preamble": 12}
        }

        # Temperature and Humidity data for plotting
        self.temp_data = []
        self.humid_data = []
        self.temp_time = []
        self.humid_time = []

        # For animated connection status
        self.connecting_movie = None

        self.init_ui()
        self.deep_sleep_timer.start(1000) # Start deep sleep monitoring check

    def init_ui(self):
        """
        Initializes and sets up the user interface elements.
        """
        # Initialize connect_btn BEFORE calling refresh_ports
        self.connect_btn = QPushButton("🔗 Connect")
        self.connect_btn.clicked.connect(self.toggle_connection)

        self.port_combo = QComboBox()
        self.refresh_ports() # Now connect_btn exists when this is called

        self.status_label = QLabel("🔌 Not Connected")
        self.status_label.setStyleSheet("color: red")

        # Load connection animation GIF
        # Menggunakan jalur relatif langsung karena di Jupyter __file__ tidak didefinisikan
        self.connecting_movie = QMovie("connecting.gif") # <--- Pastikan 'connecting.gif' di folder yang sama dengan .ipynb
        if not self.connecting_movie.isValid():
            print("Warning: connecting.gif not found or invalid. Connection status will be text-only.")
            self.connecting_movie = None

        # LoRa Configuration Group
        lora_config_group = QGroupBox("LoRa Configuration")
        lora_layout = QHBoxLayout()
        self.lora_profile_combo = QComboBox()
        self.lora_profile_combo.addItem("Select a Profile")
        for profile_name in self.lora_profiles.keys():
            self.lora_profile_combo.addItem(profile_name)
        
        self.apply_lora_btn = QPushButton("⚙ Apply Selected Profile")
        self.apply_lora_btn.clicked.connect(self.apply_lora_profile)
        
        self.save_sd_lora_btn = QPushButton("💾 Save LoRa Config to SD")
        self.save_sd_lora_btn.clicked.connect(self.save_to_sd)

        self.load_sd_lora_btn = QPushButton("🔄 Load LoRa Config from SD")
        self.load_sd_lora_btn.clicked.connect(self.load_from_sd)

        lora_layout.addWidget(QLabel("Select Profile:"))
        lora_layout.addWidget(self.lora_profile_combo)
        lora_layout.addWidget(self.apply_lora_btn)
        lora_layout.addWidget(self.save_sd_lora_btn)
        lora_layout.addWidget(self.load_sd_lora_btn)
        lora_config_group.setLayout(lora_layout)

        # Deep Sleep Configuration Group
        deep_sleep_group = QGroupBox("Deep Sleep Configuration")
        deep_sleep_layout = QHBoxLayout()
        self.deep_sleep_enable_checkbox = QComboBox() # Changed to QComboBox for clear states
        self.deep_sleep_enable_checkbox.addItem("Disable Deep Sleep Monitoring")
        self.deep_sleep_enable_checkbox.addItem("Enable Deep Sleep Monitoring")
        self.deep_sleep_enable_checkbox.setCurrentIndex(0)
        self.deep_sleep_enable_checkbox.currentIndexChanged.connect(self.toggle_deep_sleep_monitoring)

        self.deep_sleep_timeout_input = QSpinBox()
        self.deep_sleep_timeout_input.setRange(60, 3600)
        self.deep_sleep_timeout_input.setSingleStep(60)
        self.deep_sleep_timeout_input.setValue(self.deep_sleep_timeout_seconds)
        self.deep_sleep_timeout_input.setSuffix(" seconds")
        self.deep_sleep_timeout_input.setToolTip("Timeout for deep sleep (in seconds)")
        self.deep_sleep_timeout_input.valueChanged.connect(self.update_deep_sleep_timeout)

        self.deep_sleep_force_btn = QPushButton("🌙 Force Deep Sleep")
        self.deep_sleep_force_btn.clicked.connect(self.force_deep_sleep)

        deep_sleep_layout.addWidget(QLabel("Deep Sleep:"))
        deep_sleep_layout.addWidget(self.deep_sleep_enable_checkbox)
        deep_sleep_layout.addWidget(QLabel("Timeout:"))
        deep_sleep_layout.addWidget(self.deep_sleep_timeout_input)
        deep_sleep_layout.addWidget(self.deep_sleep_force_btn)
        deep_sleep_group.setLayout(deep_sleep_layout)

        # SD Card Information Group
        sd_card_group = QGroupBox("Informasi SD Card")
        sd_card_layout = QVBoxLayout()

        sd_card_buttons_layout = QHBoxLayout()
        self.list_sd_files_btn = QPushButton("📄 Daftar File SD")
        self.list_sd_files_btn.clicked.connect(self.request_sd_files)
        self.check_sd_space_btn = QPushButton("📊 Sisa Ruang SD")
        self.check_sd_space_btn.clicked.connect(self.request_sd_space)

        sd_card_buttons_layout.addWidget(self.list_sd_files_btn)
        sd_card_buttons_layout.addWidget(self.check_sd_space_btn)

        self.sd_info_display = QTextEdit()
        self.sd_info_display.setReadOnly(True)
        self.sd_info_display.setPlaceholderText("Informasi SD Card akan muncul di sini...")
        self.sd_info_display.setMinimumHeight(100)

        sd_card_layout.addLayout(sd_card_buttons_layout)
        sd_card_layout.addWidget(self.sd_info_display)
        sd_card_group.setLayout(sd_card_layout)

        self.data_display = QTextEdit()
        self.data_display.setReadOnly(True)

        self.graph_area = QWidget()
        self.graph_layout = QVBoxLayout()
        self.graph_area.setLayout(self.graph_layout)

        # Map display with loading indicator
        self.map_tab_widget = QWidget()
        self.map_tab_layout = QVBoxLayout(self.map_tab_widget)
        self.web_map = QWebEngineView()
        self.map_loading_label = QLabel("Memuat Peta...") # Map loading indicator
        self.map_loading_label.setAlignment(Qt.AlignCenter)
        self.map_loading_label.hide() # Initially hidden
        self.map_tab_layout.addWidget(self.map_loading_label)
        self.map_tab_layout.addWidget(self.web_map)

        self.web_map.loadStarted.connect(self.map_load_started)
        self.web_map.loadFinished.connect(self.map_load_finished)
        self.update_map()


        # Temperature and Humidity Tab
        self.temp_humid_graph_widget = QWidget()
        self.temp_humid_graph_layout = QVBoxLayout()
        self.temp_humid_graph_widget.setLayout(self.temp_humid_graph_layout)

        # Temperature Plot
        self.temp_plot = pg.PlotWidget(title="Temperature (°C)")
        self.temp_plot.setMinimumHeight(200)
        self.temp_plot.setBackground('k' if self.theme_dark else 'w')
        self.temp_plot.showGrid(x=True, y=True)
        self.temp_curve = self.temp_plot.plot([], [], pen=pg.mkPen('red', width=2))
        # Add fill area for temperature
        self.temp_fill_area = pg.FillBetweenItem(curve1=self.temp_curve, curve2=self.temp_plot.plot([], [], pen=None), brush=pg.mkBrush(QColor(255, 0, 0, 50)))
        self.temp_plot.addItem(self.temp_fill_area)
        self.temp_humid_graph_layout.addWidget(self.temp_plot)

        # Humidity Plot
        self.humid_plot = pg.PlotWidget(title="Humidity (%)")
        self.humid_plot.setMinimumHeight(200)
        self.humid_plot.setBackground('k' if self.theme_dark else 'w')
        self.humid_plot.showGrid(x=True, y=True)
        self.humid_curve = self.humid_plot.plot([], [], pen=pg.mkPen('blue', width=2))
        # Add fill area for humidity
        self.humid_fill_area = pg.FillBetweenItem(curve1=self.humid_curve, curve2=self.humid_plot.plot([], [], pen=None), brush=pg.mkBrush(QColor(0, 0, 255, 50)))
        self.humid_plot.addItem(self.humid_fill_area)
        self.temp_humid_graph_layout.addWidget(self.humid_plot)

        # --- New: Satellite Animation Tab ---
        self.satellite_animation_widget = QWidget()
        self.satellite_animation_layout = QVBoxLayout(self.satellite_animation_widget)
        self.web_satellite_animation = QWebEngineView()
        
        # Load the local HTML file for satellite animation
        # Menggunakan os.getcwd() karena __file__ tidak didefinisikan di lingkungan seperti Jupyter
        current_working_dir = os.getcwd() 
        html_file_path = os.path.join(current_working_dir, 'satellite_animation.html')
        
        # Periksa apakah file HTML ada sebelum memuatnya
        if os.path.exists(html_file_path):
            self.web_satellite_animation.load(QUrl.fromLocalFile(html_file_path))
        else:
            print(f"Error: satellite_animation.html not found at {html_file_path}")
            # Anda bisa menampilkan pesan error di UI jika mau
            error_label = QLabel(f"Error: satellite_animation.html tidak ditemukan di:\n{html_file_path}")
            error_label.setAlignment(Qt.AlignCenter)
            self.satellite_animation_layout.addWidget(error_label)

        self.satellite_animation_layout.addWidget(self.web_satellite_animation)
        # --- End New ---


        self.tabs = QTabWidget()
        self.tabs.addTab(self.data_display, "📡 Serial Monitor")
        self.tabs.addTab(self.graph_area, "📈 Grafik")
        self.tabs.addTab(self.temp_humid_graph_widget, "🌡️ Kelembapan & Suhu") # New tab
        self.tabs.addTab(self.map_tab_widget, "🗺️ Peta Lokasi") # Using map_tab_widget
        self.tabs.addTab(self.sd_info_display, "🗄️ SD Card Info")
        self.tabs.addTab(self.satellite_animation_widget, "🌍 Animasi Satelit") # New tab for satellite animation

        self.cmd_buttons = QHBoxLayout()
        for i in range(1, 10):
            btn = QPushButton(f"CMD {i}")
            btn.clicked.connect(lambda _, x=str(i): self.serial_reader.send(x))
            self.cmd_buttons.addWidget(btn)

        self.ctrl_layout = QHBoxLayout()
        self.auto_btn = QPushButton("Auto")
        self.auto_btn.clicked.connect(self.enable_auto)
        self.stop_btn = QPushButton("Stop")
        self.stop_btn.clicked.connect(self.disable_auto)
        self.send_file_btn = QPushButton("📁 Send File")
        self.send_file_btn.clicked.connect(self.send_file_generic) # Renamed to avoid confusion with OTA
        self.ota_btn = QPushButton("⚙ OTA Update")
        self.ota_btn.clicked.connect(self.ota_update)
        self.theme_btn = QPushButton("🎨 Toggle Theme")
        self.theme_btn.clicked.connect(self.toggle_theme)

        # OTA Progress Bar (replaces simple label)
        self.ota_progress_bar = QProgressBar()
        self.ota_progress_bar.setAlignment(Qt.AlignCenter)
        self.ota_progress_bar.setFormat("OTA Progress: %p%") # Display percentage
        self.ota_progress_bar.hide() # Hide until OTA starts

        for btn in [self.auto_btn, self.stop_btn, self.send_file_btn, self.ota_btn, self.theme_btn]:
            self.ctrl_layout.addWidget(btn)

        self.comment_layout = QHBoxLayout()
        self.comment_input = QLineEdit()
        self.comment_input.setPlaceholderText("Ketik komentar atau perintah khusus...")
        self.comment_send_btn = QPushButton("Kirim")
        self.comment_send_btn.clicked.connect(self.send_comment)
        self.comment_layout.addWidget(self.comment_input)
        self.comment_layout.addWidget(self.comment_send_btn)

        self.save_btn = QPushButton("💾 Save Log to Computer")
        self.save_btn.clicked.connect(self.save_log)

        layout = QVBoxLayout()
        connection_layout = QHBoxLayout()
        connection_layout.addWidget(QLabel("Serial Port:"))
        connection_layout.addWidget(self.port_combo)
        connection_layout.addWidget(self.connect_btn)
        connection_layout.addWidget(self.status_label)
        connection_layout.addStretch()

        layout.addLayout(connection_layout)
        layout.addWidget(lora_config_group)
        layout.addWidget(deep_sleep_group)
        layout.addWidget(sd_card_group)
        layout.addWidget(self.tabs)
        layout.addLayout(self.cmd_buttons)
        layout.addLayout(self.ctrl_layout)
        layout.addWidget(self.ota_progress_bar) # Add OTA progress bar
        layout.addLayout(self.comment_layout)
        layout.addWidget(self.save_btn)

        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)

    def refresh_ports(self):
        """
        Populates the serial port combo box with available COM ports.
        """
        self.port_combo.clear()
        ports = serial.tools.list_ports.comports()
        if not ports:
            self.port_combo.addItem("No COM Ports Found")
            self.connect_btn.setEnabled(False)
        else:
            for port in ports:
                self.port_combo.addItem(port.device)
            self.connect_btn.setEnabled(True)

    def toggle_connection(self):
        """
        Connects to or disconnects from the selected serial port.
        Includes animated connection status.
        """
        if self.connect_btn.text() == "🔗 Connect":
            port = self.port_combo.currentText()
            if port == "No COM Ports Found":
                QMessageBox.warning(self, "Connection Error", "No serial ports available.")
                return

            # Start connecting animation
            self.status_label.setText("Connecting...")
            if self.connecting_movie:
                self.status_label.setMovie(self.connecting_movie)
                self.connecting_movie.start()
            self.status_label.setStyleSheet("color: orange") # Indicate pending state
            self.update_animation_connection_status("connecting") # Update satellite animation

            # Connect in a separate thread to avoid freezing UI
            threading.Thread(target=self._attempt_connection_threaded, args=(port,), daemon=True).start()

        else:
            self.serial_reader.disconnect()
            self.connect_btn.setText("🔗 Connect")
            self.status_label.setText("🔌 Not Connected")
            self.status_label.setStyleSheet("color: red")
            if self.connecting_movie and self.connecting_movie.isRunning():
                self.connecting_movie.stop()
                self.status_label.setMovie(None) # Clear movie
            self.auto_timer.stop()
            self.deep_sleep_timer.stop()
            self.deep_sleep_enable_checkbox.setCurrentIndex(0) # Reset deep sleep monitoring UI
            self.update_animation_connection_status("disconnected") # Update satellite animation

    def _attempt_connection_threaded(self, port):
        """Helper to run connection attempt in a thread and update UI back on main thread."""
        success = self.serial_reader.connect(port)
        # Use QTimer.singleShot to update UI on the main thread
        QTimer.singleShot(0, lambda: self._update_connection_ui(success, port))

    def _update_connection_ui(self, success, port):
        """Updates the UI after a connection attempt."""
        if self.connecting_movie and self.connecting_movie.isRunning():
            self.connecting_movie.stop()
            self.status_label.setMovie(None) # Clear movie

        if success:
            self.connect_btn.setText("❌ Disconnect")
            self.status_label.setText(f"✅ Connected to {port}")
            self.status_label.setStyleSheet("color: green")
            self.update_last_activity()
            if self.deep_sleep_enabled:
                self.deep_sleep_timer.start(1000)
            self.update_animation_connection_status("connected") # Update satellite animation
        else:
            self.status_label.setText("❌ Connection Failed")
            self.status_label.setStyleSheet("color: red")
            self.log_text(f"[ERROR] Could not connect to {port}", "red")
            self.update_animation_connection_status("disconnected") # Update satellite animation

    def update_animation_connection_status(self, status):
        """
        Calls JavaScript function in the satellite animation web view to update connection status.
        """
        # Ensure the page is loaded before trying to run JavaScript
        if self.web_satellite_animation.page():
            js_command = f"window.updateConnectionStatus('{status}');"
            self.web_satellite_animation.page().runJavaScript(js_command)
            # print(f"JS command executed: {js_command}") # For debugging

    def update_animation_gcs_location(self, lat, lon):
        """
        Calls JavaScript function in the satellite animation web view to update GCS location.
        (Note: The current JS implementation uses a fixed GCS point for simplicity)
        """
        if self.web_satellite_animation.page():
            js_command = f"window.updateGCSLocation({lat}, {lon});"
            self.web_satellite_animation.page().runJavaScript(js_command)

    def handle_data(self, data):
        """
        Processes incoming data from the serial port, updates UI, and plots graphs.
        """
        timestamp = time.strftime("%H:%M:%S")
        self.log.append(f"[{timestamp}] RECV: {data}")
        self.log_text(f"[RECV] {timestamp} → {data}", color="black", highlight_duration_ms=1000) # Added highlight
        self.update_last_activity()

        # Parse specific SD Card info
        if data.startswith("SD_FILES:"):
            file_list_str = data[len("SD_FILES:"):].strip()
            self.sd_info_display.append(f"--- Daftar File SD Card ({timestamp}) ---")
            if file_list_str.startswith("ERROR:"):
                self.sd_info_display.append(f"Error: {file_list_str[6:]}")
            elif file_list_str: # Check if not empty
                for f in file_list_str.split(','):
                    self.sd_info_display.append(f"- {f.strip()}")
            else:
                self.sd_info_display.append("Tidak ada file ditemukan.")
            self.sd_info_display.append("------------------------------------------")
            self.tabs.setCurrentWidget(self.sd_info_display)
        elif data.startswith("SD_SPACE:"):
            space_info = data[len("SD_SPACE:"):].strip()
            self.sd_info_display.append(f"--- Sisa Ruang SD Card ({timestamp}) ---")
            self.sd_info_display.append(space_info)
            self.sd_info_display.append("------------------------------------------")
            self.tabs.setCurrentWidget(self.sd_info_display)
        elif data.startswith("OTA:"):
            self.log_text(f"[OTA] {data}", "darkorange", highlight_duration_ms=2000)
            if "Starting update" in data:
                self.ota_progress_bar.show()
                self.ota_progress_bar.setValue(0)
                self.ota_progress_bar.setFormat("OTA Progress: 0%")
            elif "Update successful!" in data or "Update failed:" in data:
                # Handled by update_ota_progress now
                pass


        # Existing parsing logic for general numerical data (e.g., RSSI)
        rssi_match = re.search(r"RSSI\s*[:=]\s*(-?\d+)", data)
        if rssi_match:
            self.update_graph("RSSI", int(rssi_match.group(1)))

        # Parse GPS data
        gps_match = re.search(r"LAT\s*[:=]\s*(-?\d+\.\d+).*?LON\s*[:=]\s*(-?\d+\.\d+)", data)
        if gps_match:
            self.map_lat = float(gps_match.group(1))
            self.map_lon = float(gps_match.group(2))
            self.update_map()
            self.update_animation_gcs_location(self.map_lat, self.map_lon) # Update GCS location in animation

        try:
            parsed = json.loads(data)
            if isinstance(parsed, dict):
                for key, value in parsed.items():
                    if isinstance(value, (int, float)):
                        # Only update generic graphs for keys not explicitly handled by specific plots
                        if key not in ["temperature", "humidity"]:
                            self.update_graph(key, value)
                
                # Parse temperature and humidity from JSON
                if "temperature" in parsed and isinstance(parsed["temperature"], (int, float)):
                    self.update_temp_humid_graph("temperature", parsed["temperature"])
                if "humidity" in parsed and isinstance(parsed["humidity"], (int, float)):
                    self.update_temp_humid_graph("humidity", parsed["humidity"])

        except json.JSONDecodeError:
            pass # Data is not JSON, might be a plain text message or another format

    def update_graph(self, key, value):
        """
        Updates a generic plot with new data. Creates the plot if it doesn't exist.
        """
        if key not in self.graphs:
            plot = pg.PlotWidget(title=key)
            plot.setMinimumHeight(200)
            plot.setBackground('k' if self.theme_dark else 'w')
            plot.showGrid(x=True, y=True)
            pen = pg.mkPen('c' if self.theme_dark else 'b', width=2)
            line = plot.plot([], [], pen=pen)
            
            # Add fill area for generic plots
            brush_color = QColor(0, 200, 200, 50) if self.theme_dark else QColor(0, 0, 255, 50)
            fill_area = pg.FillBetweenItem(curve1=line, curve2=plot.plot([], [], pen=None), brush=pg.mkBrush(brush_color))
            plot.addItem(fill_area)

            self.graphs[key] = {'widget': plot, 'data': [], 'time': [], 'line': line, 'fill_area': fill_area}
            self.graph_layout.addWidget(plot)
        
        g = self.graphs[key]
        now = time.time()
        # Use the global self.start_time for relative time
        g['time'].append(now - self.start_time) 
        g['data'].append(value)
        # Keep only the last 100 data points
        if len(g['data']) > 100:
            g['data'] = g['data'][-100:]
            g['time'] = g['time'][-100:]
        g['line'].setData(g['time'], g['data'])
        g['widget'].enableAutoRange('xy', True)

    def update_temp_humid_graph(self, key, value):
        """
        Updates the dedicated Temperature and Humidity plots.
        """
        now = time.time()
        # Always use self.start_time as the reference for elapsed time
        elapsed_time = now - self.start_time

        if key == "temperature":
            self.temp_time.append(elapsed_time)
            self.temp_data.append(value)
            if len(self.temp_data) > 100:
                self.temp_data = self.temp_data[-100:]
                self.temp_time = self.temp_time[-100:]
            self.temp_curve.setData(self.temp_time, self.temp_data)
            self.temp_plot.enableAutoRange('xy', True)
        elif key == "humidity":
            self.humid_time.append(elapsed_time)
            self.humid_data.append(value)
            if len(self.humid_data) > 100:
                self.humid_data = self.humid_data[-100:]
                self.humid_time = self.humid_time[-100:]
            self.humid_curve.setData(self.humid_time, self.humid_data)
            self.humid_plot.enableAutoRange('xy', True)

    def update_map(self):
        """
        Updates the Folium map with the latest GPS coordinates.
        """
        fmap = folium.Map(location=[self.map_lat, self.map_lon], zoom_start=17)
        folium.Marker([self.map_lat, self.map_lon], tooltip="Picosat Location").add_to(fmap)
        data = io.BytesIO()
        fmap.save(data, close_file=False)
        self.web_map.setHtml(data.getvalue().decode())

    def map_load_started(self):
        """Called when map loading starts."""
        self.map_loading_label.show()
        self.web_map.hide()

    def map_load_finished(self, ok):
        """Called when map loading finishes."""
        self.map_loading_label.hide()
        self.web_map.show()
        if not ok:
            self.log_text("[MAP] Failed to load map. Check internet connection.", "red")

    def log_text(self, text, color="black", highlight_duration_ms=1000):
        """
        Appends text to the serial monitor display with specified color and temporary highlight.
        """
        cursor = self.data_display.textCursor()
        cursor.movePosition(QTextCursor.End)
        
        # Original format
        fmt = QTextCharFormat()
        if self.theme_dark and color == "black":
            fmt.setForeground(QColor("white"))
        else:
            fmt.setForeground(QColor(color))
        
        # Apply original format and append
        self.data_display.setCurrentCharFormat(fmt)
        self.data_display.append(text)
        
        # Get the last block (the newly added text)
        last_block = self.data_display.document().lastBlock()
        cursor.setPosition(last_block.position())
        cursor.movePosition(QTextCursor.EndOfBlock, QTextCursor.KeepAnchor)

        # Highlight format
        highlight_fmt = QTextCharFormat(fmt) # Inherit previous format
        highlight_fmt.setBackground(QColor("rgba(255, 255, 0, 100)")) # Semi-transparent yellow
        if self.theme_dark:
            highlight_fmt.setBackground(QColor("rgba(0, 255, 255, 50)")) # Cyan for dark mode

        # Apply highlight
        self.data_display.setTextCursor(cursor)
        self.data_display.setCurrentCharFormat(highlight_fmt)
        
        # Clear highlight after a delay
        QTimer.singleShot(highlight_duration_ms, lambda: self._clear_highlight(last_block, fmt))

        self.data_display.ensureCursorVisible()

    def _clear_highlight(self, block, original_format):
        """Helper to clear the highlight from a text block."""
        # Check if the block is still valid in the document
        if not block.isValid() or block.document() != self.data_display.document():
            return

        cursor = self.data_display.textCursor()
        cursor.beginEditBlock() # For atomic operation
        
        # Ensure we're operating on the correct block and its range
        cursor.setPosition(block.position())
        cursor.movePosition(QTextCursor.EndOfBlock, QTextCursor.KeepAnchor)
        
        self.data_display.setTextCursor(cursor)
        self.data_display.setCurrentCharFormat(original_format)
        cursor.endEditBlock()
        # Re-apply global style to ensure consistency
        # This part might be tricky with specific per-line formatting. 
        # For simplicity, we ensure the background is correct after highlight.
        self.data_display.setStyleSheet(self.styleSheet()) # Reapply main window style for consistency

    def send_comment(self):
        """
        Sends user-inputted command/comment to the serial port.
        """
        msg = self.comment_input.text().strip()
        if msg:
            timestamp = time.strftime("%H:%M:%S")
            if self.serial_reader.send(msg):
                self.log.append(f"[{timestamp}] SEND: {msg}")
                self.log_text(f"[SEND] {msg}", "blue", highlight_duration_ms=1000) # Added highlight
            else:
                self.log_text(f"[ERROR] Failed to send: {msg}", "red", highlight_duration_ms=2000)
            self.comment_input.clear()
            self.update_last_activity()

    def enable_auto(self):
        """
        Enables automatic sending of CMD 1 every 10 seconds.
        """
        if not (self.serial_reader.ser and self.serial_reader.ser.is_open):
            QMessageBox.warning(self, "Connection Error", "Please connect to a serial port first.")
            return
        self.auto_timer.start(10000)
        self.log_text("[AUTO] CMD 1 aktif tiap 10s", "teal")
        self.update_last_activity()

    def disable_auto(self):
        """
        Disables automatic command sending.
        """
        self.auto_timer.stop()
        self.log_text("[AUTO] Auto ping dimatikan", "gray")

    def send_file_generic(self):
        """
        Opens a file dialog and sends the selected file's content over serial.
        (Note: This is a generic file send, not for firmware OTA)
        """
        file, _ = QFileDialog.getOpenFileName(self, "Pilih file untuk dikirim")
        if file:
            # This is for generic file sending, not specifically OTA
            # For OTA, use ota_update
            self.log_text(f"[SEND FILE] Sending {file}...", "purple")
            # In a real scenario, you'd likely define a specific protocol for generic files
            # For simplicity, this example reuses send_file which is designed for OTA chunks.
            # You might want a different method if the receiving end expects different framing.
            QMessageBox.information(self, "Info", "Generic file sending functionality needs a specific protocol on the device side. This currently uses the OTA chunking mechanism, which might not be suitable for all file types.")
            # self.serial_reader.send_file(file) # Uncomment if device has generic file reception
            self.update_last_activity()

    def ota_update(self):
        """
        Initiates an Over-The-Air (OTA) firmware update by sending a binary file.
        """
        if not (self.serial_reader.ser and self.serial_reader.ser.is_open):
            QMessageBox.warning(self, "Connection Error", "Please connect to a serial port first to perform OTA update.")
            return

        file, _ = QFileDialog.getOpenFileName(self, "Pilih binary firmware (.bin)", filter="*.bin")
        if file:
            reply = QMessageBox.question(self, "OTA Update Confirmation",
                                         f"Are you sure you want to upload '{os.path.basename(file)}' to the device? This will overwrite the current firmware.",
                                         QMessageBox.Yes | QMessageBox.No)
            if reply == QMessageBox.Yes:
                self.log_text(f"[OTA] Initiating firmware upload for: {file}", "darkgreen")
                # Start the OTA process in a separate thread to keep UI responsive
                threading.Thread(target=self.serial_reader.send_file, args=(file,), daemon=True).start()
                self.ota_progress_bar.show()
                self.ota_progress_bar.setValue(0)
                self.ota_progress_bar.setFormat("OTA Progress: 0%")
                self.update_last_activity()

    def update_ota_progress(self, progress):
        """
        Updates the OTA progress bar based on the signal from SerialReader.
        """
        if progress == -1:
            self.ota_progress_bar.setValue(0)
            self.ota_progress_bar.setFormat("OTA Failed!")
            self.log_text("[OTA] Firmware update failed!", "red")
            QMessageBox.critical(self, "OTA Update Failed", "Firmware update encountered an error.")
            self.ota_progress_bar.hide() # Hide on failure
        elif progress == 100:
            self.ota_progress_bar.setValue(100)
            self.ota_progress_bar.setFormat("OTA Complete!")
            self.log_text("[OTA] Firmware update complete. Device will reboot.", "green")
            # Keep visible for a moment then hide
            QTimer.singleShot(3000, lambda: self.ota_progress_bar.hide())
            # After successful OTA, the device will likely reboot, so auto-disconnect.
            self.toggle_connection() 
        else:
            self.ota_progress_bar.setValue(progress)
            self.ota_progress_bar.setFormat(f"OTA Progress: {progress}%")
            self.ota_progress_bar.show() # Ensure it's shown during progress

    def toggle_theme(self):
        """
        Toggles between dark and light themes for the application.
        """
        self.theme_dark = not self.theme_dark
        
        # Base style for main window
        base_style = "background-color: #2e2e2e; color: white;" if self.theme_dark else ""
        self.setStyleSheet(base_style)
        
        # Specific styles for QTextEdit and QLineEdit
        text_edit_style = f"background-color: {'#1e1e1e' if self.theme_dark else 'white'}; color: {'white' if self.theme_dark else 'black'}; border: 1px solid {'#444' if self.theme_dark else '#ddd'};"
        self.data_display.setStyleSheet(text_edit_style)
        self.comment_input.setStyleSheet(text_edit_style)
        self.sd_info_display.setStyleSheet(text_edit_style)

        # Status label color
        if self.serial_reader.ser and self.serial_reader.ser.is_open:
            self.status_label.setStyleSheet("color: #00ff00;" if self.theme_dark else "color: green;")
        else:
            self.status_label.setStyleSheet("color: #ff0000;" if self.theme_dark else "color: red;")

        # OTA Progress Bar style
        self.ota_progress_bar.setStyleSheet(f"""
            QProgressBar {{
                text-align: center;
                color: {'white' if self.theme_dark else 'black'};
                background-color: {'#444' if self.theme_dark else '#e0e0e0'};
                border: 1px solid {'#666' if self.theme_dark else '#c0c0c0'};
                border-radius: 5px;
            }}
            QProgressBar::chunk {{
                background-color: {'#4CAF50' if self.theme_dark else '#4CAF50'}; /* Green for progress */
                border-radius: 5px;
            }}
        """)

        # Pyqtgraph theme update
        for key, g in self.graphs.items():
            g['widget'].setBackground('k' if self.theme_dark else 'w')
            pen = pg.mkPen('c' if self.theme_dark else 'b', width=2)
            g['line'].setPen(pen)
            # Update fill area brush
            brush_color = QColor(0, 200, 200, 50) if self.theme_dark else QColor(0, 0, 255, 50)
            g['fill_area'].setBrush(pg.mkBrush(brush_color))

        # Update theme for temperature and humidity plots
        self.temp_plot.setBackground('k' if self.theme_dark else 'w')
        self.temp_curve.setPen(pg.mkPen('red', width=2))
        self.temp_fill_area.setBrush(pg.mkBrush(QColor(255, 0, 0, 50))) # Red, semi-transparent
        
        self.humid_plot.setBackground('k' if self.theme_dark else 'w')
        self.humid_curve.setPen(pg.mkPen('blue', width=2))
        self.humid_fill_area.setBrush(pg.mkBrush(QColor(0, 0, 255, 50))) # Blue, semi-transparent

        # Update theme for group boxes and inputs
        input_widget_style = f"color: {'white' if self.theme_dark else 'black'}; background-color: {'#3e3e3e' if self.theme_dark else 'white'};"
        for widget in [self.deep_sleep_timeout_input, self.lora_profile_combo, self.deep_sleep_enable_checkbox, self.port_combo]:
            widget.setStyleSheet(input_widget_style)

        group_box_style = f"""
            QGroupBox {{
                color: {'white' if self.theme_dark else 'black'};
                border: 1px solid {'#555' if self.theme_dark else '#ccc'};
                margin-top: 1ex;
                border-radius: 5px;
            }} 
            QGroupBox::title {{ 
                subcontrol-origin: margin; 
                subcontrol-position: top center; 
                padding: 0 3px; 
                background-color: {'#2e2e2e' if self.theme_dark else 'transparent'}; 
            }}
        """
        for group in [self.findChild(QGroupBox, "Deep Sleep Configuration"), 
                      self.findChild(QGroupBox, "LoRa Configuration"),
                      self.findChild(QGroupBox, "Informasi SD Card")]:
            if group:
                group.setStyleSheet(group_box_style)

        # Tabs widget style
        self.tabs.setStyleSheet(f"""
            QTabBar::tab {{ 
                color: {'white' if self.theme_dark else 'black'}; 
                background: {'#444' if self.theme_dark else '#f0f0f0'};
                padding: 8px 15px;
                border: 1px solid {'#555' if self.theme_dark else '#ccc'};
                border-bottom: none;
                border-top-left-radius: 4px;
                border-top-right-radius: 4px;
            }} 
            QTabBar::tab:selected {{ 
                background-color: {'#666' if self.theme_dark else 'lightgray'}; 
                border-color: {'#666' if self.theme_dark else '#999'};
                margin-bottom: -1px; /* Overlap border with pane */
            }}
            QTabBar::tab:hover {{
                background-color: {'#555' if self.theme_dark else '#e0e0e0'};
            }}
            QTabWidget::pane {{ 
                border: 1px solid {'#555' if self.theme_dark else '#ccc'}; 
                background-color: {'#2e2e2e' if self.theme_dark else 'white'};
                border-radius: 5px;
            }}
        """)
        
        # Adjust button styles for better visibility in dark mode with animations
        button_style = f"""
            QPushButton {{
                background-color: {'#555' if self.theme_dark else '#f0f0f0'};
                color: {'white' if self.theme_dark else 'black'};
                border: 1px solid {'#777' if self.theme_dark else '#bbb'};
                padding: 8px 12px;
                border-radius: 5px;
                font-weight: bold;
            }}
            QPushButton:hover {{
                background-color: {'#777' if self.theme_dark else '#e0e0e0'};
                border: 1px solid {'#999' if self.theme_dark else '#999'};
                box-shadow: 0 2px 5px rgba(0,0,0,0.3); /* Subtle shadow */
            }}
            QPushButton:pressed {{
                background-color: {'#333' if self.theme_dark else '#d0d0d0'};
                border: 1px solid {'#555' if self.theme_dark else '#888'};
                transform: translate(1px, 1px); /* Slight shift for pressed effect */
                box-shadow: none;
            }}
        """
        for btn in self.findChildren(QPushButton):
            btn.setStyleSheet(button_style)
        
        # Special style for send_comment button
        self.comment_send_btn.setStyleSheet(f"""
            QPushButton {{
                background-color: {'#1E88E5' if self.theme_dark else '#42A5F5'}; /* Blue */
                color: white;
                border: none;
                padding: 8px 12px;
                border-radius: 5px;
                font-weight: bold;
            }}
            QPushButton:hover {{
                background-color: {'#1976D2' if self.theme_dark else '#2196F3'};
            }}
            QPushButton:pressed {{
                background-color: {'#1565C0' if self.theme_dark else '#1976D2'};
            }}
        """)


    def save_log(self):
        """
        Saves the serial monitor log to a text file.
        """
        file, _ = QFileDialog.getSaveFileName(self, "Save Log", "", "Text Files (*.txt)")
        if file:
            try:
                with open(file, 'w', encoding='utf-8') as f:
                    for row in self.log:
                        f.write(row + "\n")
                self.log_text(f"[SAVE] Log saved to {file}", "green")
            except Exception as e:
                self.log_text(f"[ERROR] Gagal menyimpan log: {e}", "red")

    def apply_lora_profile(self):
        """
        Applies selected LoRa profile settings to the connected device.
        """
        selected_profile_name = self.lora_profile_combo.currentText()
        if selected_profile_name == "Select a Profile":
            QMessageBox.information(self, "Info", "Please select a LoRa profile first.")
            return

        if selected_profile_name not in self.lora_profiles:
            QMessageBox.warning(self, "Error", "Selected LoRa profile not found in defined profiles.")
            return

        config = self.lora_profiles[selected_profile_name]

        # Basic validation for config values
        for key, value in config.items():
            if not isinstance(value, (int, float)):
                QMessageBox.warning(self, "Error", f"Invalid value type for {key} in profile '{selected_profile_name}'. Please ensure it's a number.")
                return
        
        if not (self.serial_reader.ser and self.serial_reader.ser.is_open):
            QMessageBox.warning(self, "Error", "Not connected to a serial port. Cannot apply LoRa settings.")
            return

        config_str = json.dumps(config)
        if self.serial_reader.send("CONFIG:" + config_str):
            self.log_text(f"[CONFIG] Applied profile '{selected_profile_name}' to ESP32: " + config_str, "orange")
            self.update_last_activity()
        else:
            self.log_text(f"[ERROR] Failed to send LoRa config.", "red")

    def save_to_sd(self):
        """
        Sends a command to the connected device to save its current configuration to SD card.
        """
        if self.serial_reader.ser and self.serial_reader.ser.is_open:
            reply = QMessageBox.question(self, "Save to SD Card",
                                         "Are you sure you want to save the current LoRa configuration to the device's SD card?",
                                         QMessageBox.Yes | QMessageBox.No)
            if reply == QMessageBox.Yes:
                if self.serial_reader.send("!SAVE_CONFIG"):
                    self.log_text("[CMD] Meminta ESP32 menyimpan konfigurasi LoRa ke SD card", "purple")
                    self.update_last_activity()
                else:
                    self.log_text("[ERROR] Failed to send save config command.", "red")
        else:
            QMessageBox.warning(self, "Save to SD Card", "Not connected to a serial port.")

    def load_from_sd(self):
        """
        Sends a command to the connected device to load configuration from SD card.
        """
        if self.serial_reader.ser and self.serial_reader.ser.is_open:
            reply = QMessageBox.question(self, "Load from SD Card",
                                         "Are you sure you want to load LoRa configuration from the device's SD card? This will overwrite current settings.",
                                         QMessageBox.Yes | QMessageBox.No)
            if reply == QMessageBox.Yes:
                if self.serial_reader.send("!LOAD_CONFIG"):
                    self.log_text("[CMD] Meminta ESP32 memuat konfigurasi LoRa dari SD card", "purple")
                    self.update_last_activity()
                else:
                    self.log_text("[ERROR] Failed to send load config command.", "red")
        else:
            QMessageBox.warning(self, "Load from SD Card", "Not connected to a serial port.")

    def request_sd_files(self):
        """
        Sends a command to the device to list files on its SD card.
        """
        if self.serial_reader.ser and self.serial_reader.ser.is_open:
            if self.serial_reader.send("!LIST_SD"):
                self.log_text("[CMD] Meminta daftar file dari SD Card...", "green")
                self.sd_info_display.clear()
                self.sd_info_display.append("Memuat daftar file dari SD Card...")
                self.tabs.setCurrentWidget(self.sd_info_display) # Switch to SD Info tab
                self.update_last_activity()
            else:
                self.log_text("[ERROR] Failed to send list SD files command.", "red")
        else:
            QMessageBox.warning(self, "SD Card Info", "Tidak terhubung ke port serial.")

    def request_sd_space(self):
        """
        Sends a command to the device to report remaining SD card space.
        """
        if self.serial_reader.ser and self.serial_reader.ser.is_open:
            if self.serial_reader.send("!CHECK_SD_SPACE"):
                self.log_text("[CMD] Meminta sisa ruang SD Card...", "green")
                self.sd_info_display.clear()
                self.sd_info_display.append("Memuat informasi sisa ruang SD Card...")
                self.tabs.setCurrentWidget(self.sd_info_display) # Switch to SD Info tab
                self.update_last_activity()
            else:
                self.log_text("[ERROR] Failed to send check SD space command.", "red")
        else:
            QMessageBox.warning(self, "SD Card Info", "Tidak terhubung ke port serial.")

    # Deep Sleep Feature Methods
    def update_last_activity(self):
        """
        Updates the timestamp of the last serial activity.
        """
        self.last_activity_time = time.time()

    def update_deep_sleep_timeout(self, value):
        """
        Updates the deep sleep timeout from the spinbox.
        """
        self.deep_sleep_timeout_seconds = value
        self.log_text(f"[DEEP SLEEP] Timeout set to {value} seconds.", "darkcyan")

    def toggle_deep_sleep_monitoring(self, index):
        """
        Enables or disables deep sleep monitoring based on combobox selection.
        """
        if index == 1: # "Enable Deep Sleep Monitoring" selected
            if self.serial_reader.ser and self.serial_reader.ser.is_open:
                self.deep_sleep_enabled = True
                self.deep_sleep_timer.start(1000) # Start checking every second
                self.update_last_activity() # Reset activity on enable
                self.log_text("[DEEP SLEEP] Automatic deep sleep monitoring ENABLED.", "darkcyan")
            else:
                QMessageBox.warning(self, "Deep Sleep", "Please connect to a serial port first to enable deep sleep monitoring.")
                self.deep_sleep_enable_checkbox.setCurrentIndex(0) # Revert to disabled
                self.deep_sleep_enabled = False
        else: # "Disable Deep Sleep Monitoring" selected
            self.deep_sleep_enabled = False
            self.deep_sleep_timer.stop()
            self.log_text("[DEEP SLEEP] Automatic deep sleep monitoring DISABLED.", "darkcyan")

    def check_deep_sleep_condition(self):
        """
        Checks if the deep sleep condition is met (no activity for timeout duration).
        If so, sends a deep sleep command to the device.
        """
        if not (self.serial_reader.ser and self.serial_reader.ser.is_open):
            return # Only check if connected

        if not self.deep_sleep_enabled:
            return # Deep sleep monitoring is disabled

        time_since_last_activity = time.time() - self.last_activity_time
        
        if time_since_last_activity >= self.deep_sleep_timeout_seconds:
            self.log_text(f"[DEEP SLEEP] No activity for {self.deep_sleep_timeout_seconds} seconds. Sending deep sleep command...", "purple")
            if self.serial_reader.send("!DEEP_SLEEP"): # Send the deep sleep command
                self.deep_sleep_timer.stop() # Stop the timer after sending command
                self.log_text("[DEEP SLEEP] Device should now be in deep sleep. Disconnecting.", "purple")
                # Automatically disconnect after sending deep sleep command
                self.toggle_connection()
                self.deep_sleep_enable_checkbox.setCurrentIndex(0) # Reset combobox to disabled
                self.deep_sleep_enabled = False # Update internal state
            else:
                self.log_text("[ERROR] Failed to send deep sleep command.", "red")

    def force_deep_sleep(self):
        """
        Manually sends a deep sleep command to the device.
        """
        if self.serial_reader.ser and self.serial_reader.ser.is_open:
            reply = QMessageBox.question(self, "Force Deep Sleep",
                                         "Are you sure you want to force the device into deep sleep?",
                                         QMessageBox.Yes | QMessageBox.No)
            if reply == QMessageBox.Yes:
                self.log_text("[DEEP SLEEP] Manually forcing device into deep sleep...", "red")
                if self.serial_reader.send("!DEEP_SLEEP"):
                    self.deep_sleep_timer.stop() # Stop the timer
                    self.log_text("[DEEP SLEEP] Device should now be in deep sleep. Disconnecting.", "purple")
                    self.toggle_connection()
                    self.deep_sleep_enable_checkbox.setCurrentIndex(0) # Reset combobox to disabled
                    self.deep_sleep_enabled = False # Update internal state
                else:
                    self.log_text("[ERROR] Failed to send force deep sleep command.", "red")
        else:
            QMessageBox.warning(self, "Deep Sleep", "Not connected to a serial port.")


if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = GCSWindow()
    window.show()
    sys.exit(app.exec_())

Error: satellite_animation.html not found at c:\Users\Alfansyah\Downloads\satellite_animation.html
